In [35]:
%matplotlib inline
import requests
import re
from bs4 import BeautifulSoup
import pandas as pd
import time
import jieba
import jieba.analyse
from matplotlib.font_manager import FontProperties  
import matplotlib.pyplot as plt

headers = {'User-Agent':'Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/56.0.2924.87 Safari/537.36'}

#获得页面链接
def get_pageurl(page_num):
        url = 'http://tieba.baidu.com/f?kw=excel&ie=utf-8&pn='
        page_list = []
        for i in range(0,page_num*50,50):
            page_list.append(url+str(i))    #获得页面（50个）的链接
        print('已获得%i个页面链接'%len(page_list))
        return page_list

#获得每个页面链接中的帖子链接
def get_url(page_list):
    url_list = []
    r = re.compile('<a href="(.*?)" title="(.*?)" target="_blank" class="j_th_tit ">.*?</a>')   
    #把所有链接全部整合到一个列表中来
    for url in page_list:
        response = requests.session().get(url,headers=headers).text
        soup = BeautifulSoup(response,"html5lib")
        mo = r.findall(str(soup))
        p = 'http://tieba.baidu.com/'
        #对每一页主页，都提取出链接
        for i in range(len(mo)):
            url_list.append(p + mo[i][0])
    print('已获得%i个帖子链接'%len(url_list))
    return url_list

#获得所有的response
def get_response(url_list):
    response_list = []
    for url in url_list:
        try:
            response = requests.session().get(url,headers=headers).text
            response_list.append(response)
        except:
            continue
    print('已获得%i个response'%len(response_list))
    return response_list

#获得主题
def get_list(response_list):
    tieba_list = []
    for response in response_list:
        inner_list = []
        try:
            soup = BeautifulSoup(response,'lxml')
            inner_list.append(soup.h1['title'])
            r_1 = re.compile('<div author=(.*?) class="louzhubiaoshi j_louzhubiaoshi">')
            inner_list.append(r_1.findall(str(soup))[0])
            r_2 = re.compile('"date":"(.*?)"')
            inner_list.append(r_2.findall(str(soup))[0])
            tieba_list.append(inner_list)
        except:
            continue
    print('已获得%i个list'%len(tieba_list))
    return tieba_list

#把主题字符串放在一起
def get_jieba(jieba_data):
    st = ''
    word_list = []
    dic = {}
    #把主题字符串放在一起
    for i in jieba_data['主题']:
        st = st + str(i)
    seg = jieba.cut(st)
    #对字符串进行分词
    for word in seg:
        word_list.append(word)
    #获得TOP20的词语
    top_list = jieba.analyse.extract_tags(st, topK=20)
    #获得TOP20词语的词频
    for item in top_list:
        dic[item] = word_list.count(item)
    return dic

if __name__ == '__main__':
    start = time.clock()     #开始计算时间
    page_list = get_pageurl(50)  #获得50页的列表链接
    url_list = get_url(page_list)  #获得50页中的所有帖子链接
    response_list = get_response(url_list)  #获得所有帖子链接的response
    tieba_list =  get_list(response_list)   #获得所有帖子的主题、发帖人和发帖时间
    data = pd.DataFrame(tieba_list,columns = ['主题','发帖人','发帖时间'])   
    data.to_excel('D:\\MyConfiguration\\cp14330\\Desktop\\百度贴吧帖子.xlsx',index = False)   #把数据存储下来
    m = get_jieba(data)
    #把数据处理到pandas里面
    jieba_pd = pd.DataFrame(m,index=[0]).T  
    jieba_pd = jieba_pd.reset_index()
    jieba_pd = jieba_pd.rename(columns = {0:'频次','index':'主题'})
    jieba_pd = jieba_pd.sort(ascending = False,columns = '频次')
    #开始用pyplot画图
    x = [i for i in jieba_pd['主题']]
    y = [i for i in jieba_pd['频次']]
    font_set = FontProperties(fname=r"c:\windows\fonts\simsun.ttc", size=15) 
    o = [i for i in range(20)]
    plt.figure(figsize=(15,8))
    plt.bar(o,y)
    ax = plt.gca()
    ax.set_xticks(o)  
    ax.set_xticklabels(x)  
    ax.set_xticklabels(x,fontproperties=font_set)
    plt.show
    end = time.clock()  #获得程序结束时间
    print('本程序共花费%f s' %(end-start))

已获得50个页面链接
已获得2487个帖子链接
已获得2487个response
已获得2473个list
本程序共花费1969.724417 s
